## **Global output results**
---


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from sklearn import preprocessing
import json
import sys
import os
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from pathlib import Path
from collections import defaultdict

sys.path.append('../../2_fit_models/dmdm')
from data_io import get_file_dir, load_animal_list, load_cv_arr, load_data
from data_labels import create_abort_mask, partition_data_by_session
from plot_model_perform import create_cv_frame_for_plotting, plot_state_Wk, plot_state_dwelltime
from plotting_utils import load_global_glmhmm_result, calc_dwell_time, flatten_list, load_glmhmm_result
from plot_animal_behav import plot_PC, plot_CC, plot_FArate

sys.path.append('../../1_preprocess_data/dmdm')
from preprocessing_utils import load_animal_eid_dict
import matplotlib.ticker as ticker

In /nfs/nhome/live/skuroda/.conda/envs/glmhmm/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /nfs/nhome/live/skuroda/.conda/envs/glmhmm/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /nfs/nhome/live/skuroda/.conda/envs/glmhmm/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /nfs/nhome/live/skuroda/.conda/envs/glmhmm/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two mi

In [3]:
# ------- setup variables -------
dnames = ['dataAllHumans', 'dataAllMiceTraining']
C = 3  # number of output types/categories
D = 1  # data (observations) dimension
labels_for_plot_y = ['CSize', 'COnset', 'Outcome +1', 'Outcome +2', 'Outcome +3', 'Outcome +4', 'Outcome +5', 'bias']

save_figures = True
figure_dir = get_file_dir().parents[1] / 'figures'
figure_dir.mkdir(parents=True, exist_ok=True)

model = 'GLM_HMM_y'
Ks = [2,4]

In [4]:
dname = 'dataAllMiceTraining'

In [5]:
regularization = None

In [6]:
data_dir =  get_file_dir().parents[1] / "data" / "dmdm" / dname / 'data_for_cluster'

In [7]:
# ------- setup path and load data -------
data_2_dir =  get_file_dir().parents[1] / "data" / "dmdm" / dname / 'data_for_cluster' / "data_by_animal"
results_2_dir = get_file_dir().parents[1] / "results" / "dmdm_individual_fit" / dname
figure_dir = get_file_dir().parents[1] / 'figures'
figure_dir.mkdir(parents=True, exist_ok=True)

# read in data and train/test split
animal_list = load_animal_list(data_2_dir / 'animal_list.npz')

print('Animals fitted: {}'.format(animal_list))

Animals fitted: ['M_AK001' 'M_AK004' 'M_AK005' 'M_AK008' 'M_IO125' 'M_IO127' 'M_IO128'
 'M_IO132' 'M_IO135' 'M_IO136' 'M_IO137' 'M_IO138' 'M_ML007' 'M_ML008']


In [8]:
# Load raw data eids
animal_list = load_animal_list(
    data_dir / 'data_by_animal' / 'animal_list.npz')
animal_eid_dict = load_animal_eid_dict(
    data_dir / 'final_animal_eid_dict.json')

In [9]:
bestK_animals = {'M_AK001':2, 'M_AK004':3, 'M_AK005':3,
              'M_AK008':3, 'M_IO125':3,  'M_IO127':3,  
              'M_IO128':2, 'M_IO132':2,  'M_IO135':2, 
              'M_IO136':2, 'M_IO137':2,  'M_IO138':3,  
              'M_ML007':3, 'M_ML008':3}

In [12]:
for animal in animal_list:
    for K in [2,3]:
        %run -i ./output_glmhmm_states_individual.py -d {dname} -a {animal} -m {model} -K {K} --no-bestK -r {regularization} -s allsessions

Saving individual GLMHMM results ...
dname: dataAllMiceTraining; animal: M_AK001; model: GLM_HMM_y; regularization: None, state_num: 2; data_input dataAllMiceTraining_allsessions
Saving individual GLMHMM results ...
dname: dataAllMiceTraining; animal: M_AK001; model: GLM_HMM_y; regularization: None, state_num: 3; data_input dataAllMiceTraining_allsessions
Saving individual GLMHMM results ...
dname: dataAllMiceTraining; animal: M_AK004; model: GLM_HMM_y; regularization: None, state_num: 2; data_input dataAllMiceTraining_allsessions
Saving individual GLMHMM results ...
dname: dataAllMiceTraining; animal: M_AK004; model: GLM_HMM_y; regularization: None, state_num: 3; data_input dataAllMiceTraining_allsessions
Saving individual GLMHMM results ...
dname: dataAllMiceTraining; animal: M_AK005; model: GLM_HMM_y; regularization: None, state_num: 2; data_input dataAllMiceTraining_allsessions
Saving individual GLMHMM results ...
dname: dataAllMiceTraining; animal: M_AK005; model: GLM_HMM_y; regul

In [13]:
# Each animal
for animal in animal_list:
    Kanimal = bestK_animals[animal]
    %run -i ./output_glmhmm_states_individual.py -d {dname} -a {animal} -m {model} -K {Kanimal} --bestK -r {regularization} -s allsessions

Saving individual GLMHMM results ...
dname: dataAllMiceTraining; animal: M_AK001; model: GLM_HMM_y; regularization: None, state_num: 2; data_input dataAllMiceTraining_allsessions
Saving individual GLMHMM results ...
dname: dataAllMiceTraining; animal: M_AK004; model: GLM_HMM_y; regularization: None, state_num: 3; data_input dataAllMiceTraining_allsessions
Saving individual GLMHMM results ...
dname: dataAllMiceTraining; animal: M_AK005; model: GLM_HMM_y; regularization: None, state_num: 3; data_input dataAllMiceTraining_allsessions
Saving individual GLMHMM results ...
dname: dataAllMiceTraining; animal: M_AK008; model: GLM_HMM_y; regularization: None, state_num: 3; data_input dataAllMiceTraining_allsessions
Saving individual GLMHMM results ...
dname: dataAllMiceTraining; animal: M_IO125; model: GLM_HMM_y; regularization: None, state_num: 3; data_input dataAllMiceTraining_allsessions
Saving individual GLMHMM results ...
dname: dataAllMiceTraining; animal: M_IO127; model: GLM_HMM_y; regul